## Script para Filtragem dos Dados
#### v 0.1

**Dados filtrados**

* Exportações
* Produção Agrícola
* Cadastro Ambiental Rural
* Mapbiomas - Alerta
* Mapbiomas - Transição
* Mapbiomas - Cobertura
* De-para Municípios

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


dire_raw = './dados_raw/'
dire_fil = './dados_filtrados/'

## --- siglas de cada estado

siglas_uf = [
    'AM', 'AC', 'RO', 'RR',
    'PA', 'MA', 'AP', 'TO', 'MT'
]


## --- filtragem dos dados de exportação

colunas_expt = [
    'Year', 'Month', 'City', 'SH4 Code',
    'SH2 Code', 'Economic Block', 'US$ FOB',
    'Net Weight'
]

exportacoes = pd.read_csv(dire_raw+'Exportacoes (COMEXSTAT)/exportacao_full.csv', chunksize=10000,
                         engine='c', usecols=colunas_expt)

expt_filtrado = pd.DataFrame(columns=colunas_expt)

for chunk in exportacoes:
    select = chunk['City'].apply(lambda cidade: cidade.split('-')[1].strip() in siglas_uf)
    expt_filtrado = pd.concat([expt_filtrado, chunk[select]])
    
expt_filtrado.to_csv(dire_fil+'exportacoes_filtrado.csv', index=False)
del expt_filtrado


## --- filtragem dos dados de produção agrícola

colunas_prod = [
    'geo_id', 'ano', 'area_colhida_v0',
    'area_plantada_dest_colheita_v0', 'valor_v0'
]

producao_agr = pd.read_excel(dire_raw+'Producao Agricola Municipal (PAM)/PAM.xlsx',
                            sheet_name='Dados', usecols=colunas_prod)

producao_agr.to_csv(dire_fil+'PAM_filtrado.csv', index=False)
del producao_agr


## --- ajuste cadastro ambiental rural

car = pd.read_csv(dire_raw+'Cadastro Ambiental Rural (CAR)/car_agregado_municipio.csv', sep=';')

car.to_csv(dire_fil+'CAR_ajustado.csv', index=False)
del car


## --- filtragem mapbiomas (alertas)

colunas_alerta = [
    'rural_property_id', 'car_code', 'car_area_ha',
    'alert_car_area_ha', 'alert_total_area_ha', 'alert_id',
    'source', 'detected_at', 'biome_id', 'state_id', 'city_id',
    'legal_reserve_area_ha', 'alert_legal_reserve_area_ha',
    'alert_car_indigenous_land_id', 'alert_car_indigenous_area_ha',
    'alert_car_pmfs_area_ha','car_embargoed_area_ha'
]

alertas = pd.read_excel(dire_raw+'Mapbiomas - alertas/DB_ALL_ALERTS_RAD2020_SITE.xlsx',
                       sheet_name='BD_ALERTAS_VAL_CAR', usecols=colunas_alerta )

select = alertas['car_code'].apply(lambda x: x.split('-')[0] in siglas_uf)
alertas = alertas[select]

alertas.to_csv(dire_fil+'alertas_filtrado.csv', index=False)
del alertas


## --- filtragem mapbiomas (transição)

colunas_transicao = [
       'territory_id', 'municipality', 'state', 'from', 'to', 'from_level_0',
       'from_level_1', 'from_level_2', 'from_level_3', 'from_level_4',
       'to_level_0', 'to_level_1', 'to_level_2', 'to_level_3', 'to_level_4',
       '2000 to 2001', '2000 to 2005', '2000 to 2010', '2000 to 2019', 
       '2001 to 2002', '2001 to 2016', '2002 to 2003', '2002 to 2010', 
       '2005 to 2006','2005 to 2010', '2008 to 2009','2008 to 2018', 
       '2008 to 2019', '2010 to 2011', '2010 to 2015', '2010 to 2016', 
       '2010 to 2019', '2011 to 2012', '2012 to 2013', '2012 to 2019', 
       '2013 to 2014', '2015 to 2016', '2015 to 2019', '2016 to 2017', 
       '2017 to 2018', '2018 to 2019'
]

transicao = pd.read_excel(dire_raw+'Mapbiomas - Transicao/Dados_Transicao_MapBiomas_5.0_UF-MUN_SITE_v2.xlsx', 
                          sheet_name=2)

select = transicao['state'].apply(lambda x: x in siglas_uf)
transicao = transicao[select]

transicao.to_csv(dire_fil+'transicao_filtrado.csv', index=False)
del transicao


## --- filtragem mapbiomas (cobertura)

cobertura = pd.read_csv(dire_raw+'Mapbiomas - Cobertura/mapbiomas_cover.csv', engine='c')
cobertura = cobertura[cobertura['year']>1999]
cobertura = cobertura[cobertura['state'].apply(lambda s: s in siglas_uf)]
cobertura.drop('municipality', axis=1, inplace=True)

cobertura.to_csv(dire_fil+'cobertura_filtrado.csv', index=False)
del cobertura

## --- filtragem de-para municipios

depara_mun = pd.read_csv(dire_raw+'UF_MUN.csv', engine='c', sep=';', encoding = "ISO-8859-1")
depara_mun = depara_mun[depara_mun['SG_UF'].apply(lambda s: s in siglas_uf)]
depara_mun.drop(['NO_MUN', 'CO_MUND_IBGE_6'], axis=1, inplace=True)

depara_mun.to_csv(dire_fil+'depara_filtrado.csv', index=False)
del depara_mun